In [1]:
trainingFile = "../data/FeaturesAsMatrixv1.csv"

In [45]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import merge
import pandas as pd
import numpy as np
from keras.models import Sequential


In [37]:
dfTrainingData.columns[60]

'5_mols'

In [4]:
dfTrainingData = pd.read_csv(trainingFile)

In [38]:
targetCols = ["%d_target"%i for i in range(0,50)]
molregnoCols =  ["%d_mols"%i for i in range(0,50)]
latentFeatCols = ["%d_latfeatures"%i for i in range(0,292)]

In [43]:
targetArr = np.asarray(dfTrainingData.as_matrix(columns=targetCols))
molregNoArr = np.asarray(dfTrainingData.as_matrix(columns=molregnoCols))
latfeatureArr = np.asarray(dfTrainingData.as_matrix(columns=latentFeatCols))


#targetArr = (dfTrainingData[targetCols])
#molregNoArr = (dfTrainingData[molregnoCols])
#latfeatureArr = (dfTrainingData[latentFeatCols])


In [9]:
### Build Sequential network

targetInput = Input?

In [121]:
targetInput = Input(shape=(50,))
x1 = Dense(100, activation='linear')(targetInput)
x1 = Dense(150, activation='linear')(x1)


molregnoInput = Input(shape=(50,))
x2 = Dense(100, activation='linear')(molregnoInput)
x2 = Dense(150, activation='linear')(x2)

mergedLayer = merge([x1,x2],concat_axis=1,mode="concat")
####merge function is depreceted - change it next revision

x = Dense(200, activation='linear')(mergedLayer)
x = Dense(250, activation='linear')(x)
outputLayer = Dense(292, activation='linear')(x)

model = Model([targetInput,molregnoInput],outputLayer)

model.compile(optimizer='adadelta', loss='mse')

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # Remove the CWD from sys.path while we load stuff.


In [122]:
train_history = model.fit([targetArr , molregNoArr], latfeatureArr,
                epochs=100,
                batch_size=256,
                shuffle=True)

Epoch 1/100
35955/35955 [==============================] - 15s 425us/step - loss: 0.0434
Epoch 2/100
35955/35955 [==============================] - 11s 311us/step - loss: 0.0139
Epoch 3/100
35955/35955 [==============================] - 11s 305us/step - loss: 0.0119
Epoch 4/100
35955/35955 [==============================] - 11s 313us/step - loss: 0.0110
Epoch 5/100
35955/35955 [==============================] - 11s 317us/step - loss: 0.0104
Epoch 6/100
35955/35955 [==============================] - 12s 322us/step - loss: 0.0100
Epoch 7/100
35955/35955 [==============================] - 11s 318us/step - loss: 0.0097
Epoch 8/100
35955/35955 [==============================] - 11s 318us/step - loss: 0.0094
Epoch 9/100
35955/35955 [==============================] - 12s 320us/step - loss: 0.0092
Epoch 10/100
35955/35955 [==============================] - 12s 323us/step - loss: 0.0090
Epoch 11/100
35955/35955 [==============================] - 12s 322us/step - loss: 0.0089
Epoch 12/100
35955/

35955/35955 [==============================] - 8s 222us/step - loss: 0.0069
Epoch 93/100
35955/35955 [==============================] - 8s 218us/step - loss: 0.0069
Epoch 94/100
35955/35955 [==============================] - 8s 216us/step - loss: 0.0069
Epoch 95/100
35955/35955 [==============================] - 8s 218us/step - loss: 0.0069
Epoch 96/100
35955/35955 [==============================] - 8s 221us/step - loss: 0.0068
Epoch 97/100
35955/35955 [==============================] - 8s 218us/step - loss: 0.0068
Epoch 98/100
35955/35955 [==============================] - 8s 219us/step - loss: 0.0068
Epoch 99/100
35955/35955 [==============================] - 8s 216us/step - loss: 0.0068
Epoch 100/100
35955/35955 [==============================] - 8s 216us/step - loss: 0.0068


In [123]:
ypred = model.predict([targetArr[:30,:],molregNoArr[:30,:]])

In [140]:
!rm temp.txt

In [88]:

### Load MVAE
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from mVAE.model import MoleculeVAE
from mVAE.utils import encode_smiles, decode_latent_molecule, interpolate, get_unique_mols
#import mVAE.mVAE
# number of dimensions to represent the molecules
# as the model was trained with this number, any operation made with the model must share the dimensions.
latent_dim = 292

# trained_model 0.99 validation accuracy
# trained with 80% of ALL chembl molecules, validated on the other 20.
trained_model = 'mVAE/chembl_23_model.h5'
charset_file = 'charset.json'

# load charset and model
with open('mVAE/charset.json', 'r') as outfile:
    charset = json.load(outfile)

modelVae = MoleculeVAE()
modelVae.load(charset, trained_model, latent_rep_size = latent_dim)


In [127]:
!ls ../data

AssayDataSetForDeeplearning.csv
Chembl23DataSetForFiltering_5andMore.csv
CompoundFingerPrints_LatentFectors_MultiBinary.csv
FeaturesAsMatrixv1.csv
TargetFingerPrints_LatentFectors_MultiBinary.csv
smilesCode.csv


In [124]:
reconstructed_aspirin = decode_latent_molecule(ypred[12,:], modelVae, charset, 292)
#original = Chem.MolFromSmiles(testSMILES)
reconstructued = Chem.MolFromSmiles(reconstructed_aspirin)

#Draw.MolsToGridImage([reconstructued])
from rdkit import RDLogger
# remove warnings and errors from notebook (lots of them due non valid molecule generation)
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

workingMols = list()
for predMols in ypred:
    decodedSmiles =  decode_latent_molecule(predMols, modelVae, charset, 292)
    reconstructued = Chem.MolFromSmiles(decodedSmiles)
    
    if reconstructued:
        print(decodedSmiles)
        workingMols.append(reconstructued)
        
print("No of successful predictions %d"%len(workingMols))

No of successful predictions 0


In [83]:
ypred[0,0:10] 

array([-0.10926732,  0.09486426,  0.14250977,  0.05274021, -0.0953891 ,
        0.06582877,  0.12305044, -0.05070004, -0.24215151,  0.12526383], dtype=float32)